In [9]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [10]:
states = ['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

In [11]:
# Example data
data = [0.757415958, 0.724739389, 0.72008117, 0.758953258, 0.730860224, 0.757255898, 0.754841234, 0.76845962, 0.77734042]

# ETS Model
ets_model = ExponentialSmoothing(data, trend='add', seasonal=None)
ets_fit = ets_model.fit()
ets_forecast = ets_fit.forecast(steps=5)

# ARIMA Model
arima_model = ARIMA(data, order=(1, 1, 1))
arima_fit = arima_model.fit()
arima_forecast = arima_fit.forecast(steps=5)

# LSTM Model
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(np.array(data).reshape(-1, 1))

# Create sequences
sequence_length = 3
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i + sequence_length])
    y.append(data_scaled[i + sequence_length])
X, y = np.array(X), np.array(y)

# Define LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X, y, epochs=50, verbose=0)

# Prepare input for forecasting
last_sequence = data_scaled[-sequence_length:].reshape(1, sequence_length, 1)
lstm_forecast_scaled = [lstm_model.predict(last_sequence, verbose=0)[0, 0] for _ in range(5)]
lstm_forecast = scaler.inverse_transform(np.array(lstm_forecast_scaled).reshape(-1, 1)).flatten()

# Simple Average
average_forecast = (ets_forecast + arima_forecast + lstm_forecast) / 3

# Weighted Average (Example weights: 0.4, 0.3, 0.3 for ETS, ARIMA, LSTM)
weights = [0.4, 0.3, 0.3]
weighted_forecast = (weights[0] * ets_forecast +
                     weights[1] * arima_forecast +
                     weights[2] * lstm_forecast)

print("ETS Forecast:", ets_forecast)
print("ARIMA Forecast:", arima_forecast)
print("LSTM Forecast:", lstm_forecast)
print("Simple Average Forecast:", average_forecast)
print("Weighted Average Forecast:", weighted_forecast)

C:\Users\anees\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\anees\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ETS Forecast: [0.77321508 0.77785942 0.78250375 0.78714809 0.79179242]
ARIMA Forecast: [0.77957185 0.77754057 0.77938965 0.77770642 0.77923868]
LSTM Forecast: [0.7390465 0.7390465 0.7390465 0.7390465 0.7390465]
Simple Average Forecast: [0.76394448 0.7648155  0.76697997 0.76796701 0.77002587]
Weighted Average Forecast: [0.76487155 0.7661199  0.76853236 0.76988512 0.77220253]
